# GP1 Trading Model

## Get the API Keys Ready

In [ ]:
from config_private import ALPACA_API_KEY, ALPACA_API_SECRET
API_BASE_URL = 'https://paper-api.alpaca.markets'

## Part 1: Imports and Class/Function definitions

In [ ]:
from config_tickers import GP1_TICKER
from config import INDICATORS
from config import CDL
from data_processor import DataProcessor
from plot import backtest_stats, backtest_plot, get_baseline, drop_dup_dates
import pickle
import numpy as np
import pandas as pd
import os

## Environment

In [ ]:
from env_stocktrading_np import StockTradingEnv

## DRL Agent Class

In [ ]:
from DRLagent import DRLAgent

## Train Function

In [ ]:
def train(
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):

    # Create 'train_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "train_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'train_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
            
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": True,
    }
    env_instance = env(config=env_config)

    # read parameters
    cwd = kwargs.get("cwd", "./" + str(model_name))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        break_step = kwargs.get("break_step", 1e6)
        erl_params = kwargs.get("erl_params")
        agent = DRLAgent_erl(
            env=env,
            price_array=price_array,
            tech_array=tech_array,
            turbulence_array=turbulence_array,
            date_array=date_array,
        )
        model = agent.get_model(model_name, model_kwargs=erl_params)
        trained_model = agent.train_model(
            model=model, cwd=cwd, total_timesteps=break_step
        )    

## Test Function

In [ ]:
def test(
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):

    # Create 'test_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "test_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'test_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
            
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": False,
    }
    env_instance = env(config=env_config)

    # load elegantrl needs state dim, action dim and net dim
    net_dimension = kwargs.get("net_dimension", 2**7)
    cwd = kwargs.get("cwd", "./" + str(model_name))
    print("price_array: ", len(price_array))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        episode_dates, episode_total_assets, _, num_trades = DRLAgent_erl.DRL_prediction(
            model_name=model_name,
            cwd=cwd,
            net_dimension=net_dimension,
            environment=env_instance,
        )
        
        print(f"Number of trades made: {num_trades}")
        episode_dates = np.unique(np.concatenate(episode_dates))
        episode_dates = np.concatenate(([episode_dates[0]], episode_dates))
        episode_dates = [timestamp.date() for timestamp in episode_dates]
        account_value_df = pd.DataFrame({'date': episode_dates, 'account_value': episode_total_assets})
        return account_value_df

## Import Stock Symbols and Indicators

In [ ]:
ticker_list = GP1_TICKER
action_dim = len(GP1_TICKER)

In [ ]:
print(ticker_list)

In [ ]:
print(INDICATORS)

In [ ]:
print(CDL)

## Calculate the DRL State Dimension Manually for Trading

In [ ]:
state_dim = 109

In [ ]:
state_dim

# Part 2: Train the Agent

Note: Start Date Must Be Monday for backtest plots

## Hyperparameter optimization Functions

In [ ]:
import optuna
import random
from datetime import datetime
from calendar import monthrange


env = StockTradingEnv

TRAIN_START_DATE = "2016-01-01"
TRAIN_END_DATE = "2016-01-31"

def optuna_train(
    trial,
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):
    # Create 'train_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "train_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'train_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
        
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": True,
    }
    env_instance = env(config=env_config)

    # read parameters
    cwd = kwargs.get("cwd", "./" + str(model_name))
    
    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        break_step = kwargs.get("break_step", 1e6)
        erl_params = kwargs.get("erl_params")
        agent = DRLAgent_erl(
            env=env,
            price_array=price_array,
            tech_array=tech_array,
            turbulence_array=turbulence_array,
            date_array=date_array,
        )
        model = agent.get_model(model_name, model_kwargs=erl_params)
        trained_model = agent.train_model(
            model=model, cwd=cwd, total_timesteps=break_step
        )

def optuna_test(
    trial,
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):
    # Create 'test_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "test_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'test_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": False,
    }
    env_instance = env(config=env_config)

    # load elegantrl needs state dim, action dim and net dim
    net_dimension = kwargs.get("net_dimension")
    cwd = kwargs.get("cwd", "./" + str(model_name))
    print("price_array: ", len(price_array))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        episode_dates, episode_total_assets, _, num_trades = DRLAgent_erl.DRL_prediction(
            model_name=model_name,
            cwd=cwd,
            net_dimension=net_dimension,
            environment=env_instance,
        )
        
        print(f"Number of trades made: {num_trades}")
        episode_dates = np.unique(np.concatenate(episode_dates))
        episode_dates = np.concatenate(([episode_dates[0]], episode_dates))
        episode_dates = [timestamp.date() for timestamp in episode_dates]
        account_value_df = pd.DataFrame({'date': episode_dates, 'account_value': episode_total_assets})
        return account_value_df, num_trades


def optuna_calculate_loss(account_value_daily, w1=0.6, w2=0.4):
    pnl = account_value_daily['account_value'].iloc[-1] - account_value_daily['account_value'].iloc[0]
    neg_pnl = -pnl

    running_max = np.maximum.accumulate(account_value_daily['account_value'])
    drawdowns = (running_max - account_value_daily['account_value']) / running_max
    max_drawdown = np.max(drawdowns)

    loss = w1 * neg_pnl + w2 * max_drawdown
    return loss


def optuna_objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 0.20, log=True)
    lambda_entropy = trial.suggest_float("lambda_entropy", 1e-5, 0.20, log=True)
    batch_size = trial.suggest_categorical("batch_size", [512, 1024, 2048, 4096])
    gamma = trial.suggest_categorical("gamma", [0.985, 0.99, 0.995])
    net_dimension = []
    n_hidden_layers = trial.suggest_int("n_hidden_layers", 1, 5)
    possible_layer_sizes = [8, 16, 32, 64, 128, 256, 512, 1024, 2048]
    for i in range(n_hidden_layers):
        layer_size = trial.suggest_categorical(f"hidden_layer_{i}", possible_layer_sizes)
        net_dimension.append(layer_size)
    target_step = 5000
    eval_gap = 30
    eval_times = 1
    erl_params = {
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "lambda_entropy": lambda_entropy,
        "batch_size": batch_size,
        "gamma": gamma,
        "seed": 999,
        "net_dimension": net_dimension,
        "target_step": target_step,
        "eval_gap": eval_gap,
        "eval_times": eval_times,
    }
    
    random_year = random.randint(2017, 2022)
    random_month = random.randint(1, 12)
    _, last_day_of_month = monthrange(random_year, random_month)

    TEST_START_DATE = datetime(random_year, random_month, 1)
    TEST_END_DATE = datetime(random_year, random_month, last_day_of_month)
    TEST_START_DATE = TEST_START_DATE.strftime('%Y-%m-%d')
    TEST_END_DATE = TEST_END_DATE.strftime('%Y-%m-%d')
    
    optuna_train(
        trial,
        start_date = TRAIN_START_DATE, 
        end_date = TRAIN_END_DATE,
        ticker_list = ticker_list, 
        data_source = 'alpaca',
        time_interval = "15Min", 
        technical_indicator_list = INDICATORS,
        drl_lib ='elegantrl', 
        env = env,
        model_name ='ppo',
        if_vix = True,
        if_cdl = True, 
        API_KEY = ALPACA_API_KEY, 
        API_SECRET = ALPACA_API_SECRET, 
        API_BASE_URL = API_BASE_URL,
        cwd = './gp1_testing', #current_working_dir
        break_step = 2e5,
        erl_params = erl_params,
    )

    account_value_df, _ = optuna_test(
        trial,
        start_date = TEST_START_DATE, 
        end_date = TEST_END_DATE,
        ticker_list = ticker_list, 
        data_source = 'alpaca',
        time_interval = "15Min", 
        technical_indicator_list = INDICATORS,
        drl_lib = 'elegantrl', 
        env = env, 
        model_name = 'ppo',
        if_vix = True,
        if_cdl = True,
        API_KEY = ALPACA_API_KEY, 
        API_SECRET = ALPACA_API_SECRET, 
        API_BASE_URL = API_BASE_URL,
        cwd = './gp1_testing',
        net_dimension = erl_params['net_dimension'],
    )

    account_value_df['date'] = pd.to_datetime(account_value_df['date']).dt.date
    account_value_daily = account_value_df.groupby('date')['account_value'].last().reset_index()

    account_value_daily['daily_return'] = account_value_daily['account_value'].pct_change(1)

    if account_value_daily['daily_return'].std() > 1e-8:
        sharpe_ratio = (252**0.5) * account_value_daily['daily_return'].mean() / account_value_daily['daily_return'].std()
    else:
        sharpe_ratio = 0
    
    loss = optuna_calculate_loss(account_value_daily)

    return loss, sharpe_ratio

## Run Optuna Trials

In [ ]:
study_name = "gp1_trials"
directions = ["minimize", "maximize"]

study = optuna.create_study(study_name=study_name, directions=directions)

# timeout=3600 is 1 hr, 86400 is 24hrs or n_trials=100
study.optimize(optuna_objective, n_trials=10)

In [ ]:
# Get Pareto front trials
pareto_front_trials = study.best_trials

# Print Pareto front trials
print(f"Number of trials on the Pareto front: {len(study.best_trials)}")
for trial in pareto_front_trials:
    print(f"  Trial {trial.number}:")
    print("    Value: ", trial.values)
    print("    Params: ")
    for key, value in trial.params.items():
        print(f"      {key}: {value}")

In [ ]:
optuna.visualization.plot_pareto_front(study, target_names=["loss", "sharpe_ratio"])

In [ ]:
# Export Visualization
import plotly.io as pio


pareto_front = optuna.visualization.plot_pareto_front(study, target_names=["loss", "sharpe_ratio"])

pio.write_html(pareto_front, file='pareto_front.html', auto_open=False)


## Filter Trials

In [ ]:
trials = study.get_trials()

def trial_filter(trial):
    if trial.values is None:
        return False
    
    # order of objectives:
    # 0: loss (minimize)
    # 1: sharpe ratio (maximize)
    # 2: number of trades (minimize)
    
    # Set thresholds for each objective
    loss_threshold = 0
    sharpe_ratio_threshold = 0
    # num_trades_threshold = 100

    if (trial.values[0] == loss_threshold and 
        trial.values[1] == sharpe_ratio_threshold):
        # trial.values[2] >= num_trades_threshold
        return True
    return False

filtered_trials = [t for t in trials if trial_filter(t)]

## Filtered Trials Stats

In [ ]:
# Initialize variables for min, max, and sum
min_learning_rate = float('inf')
max_learning_rate = float('-inf')
sum_learning_rate = 0

min_weight_decay = float('inf')
max_weight_decay = float('-inf')
sum_weight_decay = 0

min_lambda_entropy = float('inf')
max_lambda_entropy = float('-inf')
sum_lambda_entropy = 0

# Iterate through the filtered trials and calculate the required statistics
num_trials = len(filtered_trials)

for trial in filtered_trials:
    learning_rate = trial.params['learning_rate']
    weight_decay = trial.params['weight_decay']
    lambda_entropy = trial.params['lambda_entropy']

    min_learning_rate = min(min_learning_rate, learning_rate)
    max_learning_rate = max(max_learning_rate, learning_rate)
    sum_learning_rate += learning_rate

    min_weight_decay = min(min_weight_decay, weight_decay)
    max_weight_decay = max(max_weight_decay, weight_decay)
    sum_weight_decay += weight_decay

    min_lambda_entropy = min(min_lambda_entropy, lambda_entropy)
    max_lambda_entropy = max(max_lambda_entropy, lambda_entropy)
    sum_lambda_entropy += lambda_entropy

# Calculate averages
avg_learning_rate = sum_learning_rate / num_trials
avg_weight_decay = sum_weight_decay / num_trials
avg_lambda_entropy = sum_lambda_entropy / num_trials

# Print the results
print("Learning rate: min={}, max={}, avg={}".format(min_learning_rate, max_learning_rate, avg_learning_rate))
print("Weight decay: min={}, max={}, avg={}".format(min_weight_decay, max_weight_decay, avg_weight_decay))
print("Lambda entropy: min={}, max={}, avg={}".format(min_lambda_entropy, max_lambda_entropy, avg_lambda_entropy))

## The Gauntlet

In [ ]:
import os
import pickle
import shutil
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY
from dateutil.parser import parse
from calendar import monthrange

import optuna
import re
import glob


env = StockTradingEnv

TRAIN_START_DATE = "2016-01-01"
TRAIN_END_DATE = "2016-12-31"
TEST_START_DATE = "2017-01-01"
TEST_END_DATE = "2017-01-31"
END_DATE = '2023-03-31'

def optuna_train(
    trial,
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):
    # Create 'train_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "train_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'train_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
        
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": True,
    }
    env_instance = env(config=env_config)

    # read parameters
    cwd = kwargs.get("cwd", "./" + str(model_name))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        break_step = kwargs.get("break_step", 1e6)
        erl_params = kwargs.get("erl_params")
        agent = DRLAgent_erl(
            env=env,
            price_array=price_array,
            tech_array=tech_array,
            turbulence_array=turbulence_array,
            date_array=date_array,
        )
        model = agent.get_model(model_name, model_kwargs=erl_params)
        trained_model = agent.train_model(
            model=model, cwd=cwd, total_timesteps=break_step
        )

def optuna_test(
    trial,
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):
    # Create 'test_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "test_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'test_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
            
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": False,
    }
    env_instance = env(config=env_config)

    # load elegantrl needs state dim, action dim and net dim
    net_dimension = kwargs.get("net_dimension")
    cwd = kwargs.get("cwd", "./" + str(model_name))
    print("price_array: ", len(price_array))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        episode_dates, episode_total_assets, episode_return, num_trades = DRLAgent_erl.DRL_prediction(
            model_name=model_name,
            cwd=cwd,
            net_dimension=net_dimension,
            environment=env_instance,
        )
        
        print(f"Number of trades made: {num_trades}")
        episode_dates = np.unique(np.concatenate(episode_dates))
        episode_dates = np.concatenate(([episode_dates[0]], episode_dates))
        episode_dates = [timestamp.date() for timestamp in episode_dates]
        account_value_df = pd.DataFrame({'date': episode_dates, 'account_value': episode_total_assets})
        return account_value_df, episode_return, num_trades


def optuna_calculate_loss(account_value_daily, w1=0.6, w2=0.4):
    pnl = account_value_daily['account_value'].iloc[-1] - account_value_daily['account_value'].iloc[0]
    neg_pnl = -pnl

    running_max = np.maximum.accumulate(account_value_daily['account_value'])
    drawdowns = (running_max - account_value_daily['account_value']) / running_max
    max_drawdown = np.max(drawdowns)

    loss = w1 * neg_pnl + w2 * max_drawdown
    return loss

def optuna_gauntlet_objective(trial):
    returns_threshold = 0.9
    monthly_returns = []
    monthly_loss = []
    all_test_daily_returns = pd.DataFrame()
    train_start_date = datetime.strptime(TRAIN_START_DATE, '%Y-%m-%d').date()
    train_end_date = datetime.strptime(TRAIN_END_DATE, '%Y-%m-%d').date()
    test_start_date = datetime.strptime(TEST_START_DATE, '%Y-%m-%d').date()
    test_end_date = datetime.strptime(TEST_END_DATE, '%Y-%m-%d').date()
    end_date = datetime.strptime(END_DATE, '%Y-%m-%d').date()
    
    while train_end_date <= end_date:
        learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True)
        weight_decay = trial.suggest_float("weight_decay", 1e-5, 0.20, log=True)
        lambda_entropy = trial.suggest_float("lambda_entropy", 1e-5, 0.20, log=True)
        batch_size = trial.suggest_categorical("batch_size", [512, 1024, 2048, 4096])
        gamma = trial.suggest_categorical("gamma", [0.985, 0.99, 0.995])
        net_dimension = []
        n_hidden_layers = trial.suggest_int("n_hidden_layers", 1, 5)
        possible_layer_sizes = [8, 16, 32, 64, 128, 256, 512, 1024, 2048]
        for i in range(n_hidden_layers):
            layer_size = trial.suggest_categorical(f"hidden_layer_{i}", possible_layer_sizes)
            net_dimension.append(layer_size)
        target_step = 5000
        eval_gap = 30
        eval_times = 1
        erl_params = {
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "lambda_entropy": lambda_entropy,
            "batch_size": batch_size,
            "gamma": gamma,
            "seed": 999,
            "net_dimension": net_dimension,
            "target_step": target_step,
            "eval_gap": eval_gap,
            "eval_times": eval_times,
        }
        
        optuna_train(
            trial,
            start_date=train_start_date.strftime('%Y-%m-%d'),
            end_date=train_end_date.strftime('%Y-%m-%d'),
            ticker_list = ticker_list, 
            data_source = 'alpaca',
            time_interval= "15Min", 
            technical_indicator_list= INDICATORS,
            drl_lib='elegantrl', 
            env=env,
            model_name='ppo',
            if_vix=True,
            if_cdl=True, 
            API_KEY = ALPACA_API_KEY, 
            API_SECRET = ALPACA_API_SECRET, 
            API_BASE_URL = API_BASE_URL,
            cwd='./gp1_testing', #current_working_dir
            break_step=1e6,
            erl_params=erl_params
        )

        account_value_df, episode_return, _ = optuna_test(
            trial,
            start_date=test_start_date.strftime('%Y-%m-%d'),
            end_date=test_end_date.strftime('%Y-%m-%d'),
            ticker_list = ticker_list, 
            data_source = 'alpaca',
            time_interval= "15Min", 
            technical_indicator_list= INDICATORS,
            drl_lib='elegantrl', 
            env=env, 
            model_name='ppo',
            if_vix=True,
            if_cdl=True,
            API_KEY = ALPACA_API_KEY, 
            API_SECRET = ALPACA_API_SECRET, 
            API_BASE_URL = API_BASE_URL,
            cwd='./gp1_testing',
            net_dimension = erl_params['net_dimension'],
        )
        # Stats
        account_value_df['date'] = pd.to_datetime(account_value_df['date']).dt.date
        account_value_daily = account_value_df.groupby('date')['account_value'].last().reset_index()
        account_value_daily['daily_return'] = account_value_daily['account_value'].pct_change(1)

        monthly_return = episode_return
        
        # Check if all thresholds are met
        if (
            monthly_return >= returns_threshold
        ):
            # Record Stats
            loss = optuna_calculate_loss(account_value_daily)
            monthly_returns.append(monthly_return)
            monthly_loss.append(loss)
            all_test_daily_returns = pd.concat([all_test_daily_returns, account_value_daily[['date', 'daily_return']]], ignore_index=True)
            
            # Move the date range one month forward
            train_start_date = test_start_date
            train_end_date = test_end_date
            test_start_date = list(rrule(freq=MONTHLY, dtstart=parse(test_start_date.strftime('%Y-%m-%d')), count=2))[-1].date()
            _, last_day_of_month = monthrange(test_start_date.year, test_start_date.month)
            test_end_date = test_start_date.replace(day=last_day_of_month)
        else:
            # Delete the model if saved to disk
            if os.path.exists("./gp1_testing/actor.pth"):
                os.remove("./gp1_testing/actor.pth")
            # Print the hyperparameters
            print(f"Trial {trial.number} pruned. Hyperparameters: learning_rate={learning_rate}, weight_decay={weight_decay}, lambda_entropy={lambda_entropy}, net_dimension={net_dimension}")
            # Cancel Trial
            raise optuna.TrialPruned()
        
    total_trading_days = len(all_test_daily_returns)
    annualized_factor = np.sqrt(total_trading_days)
    sharpe_ratio = annualized_factor * all_test_daily_returns['daily_return'].mean() / all_test_daily_returns['daily_return'].std()
    cumulative_monthly_loss = np.sum(monthly_loss)
    cumulative_monthly_return = np.prod(monthly_returns)
    
    # Move and rename the model file
    src_file = "./gp1_testing/actor.pth"
    dst_dir = "./gp1_gauntlet"
    os.makedirs(dst_dir, exist_ok=True)
    dst_file = os.path.join(dst_dir, f"actor_trial_{trial.number}_return_{format(round(cumulative_monthly_return, 2), '.2f').replace('.', '_')}.pth")
    shutil.move(src_file, dst_file)
    
    # Get the list of all actor.pth files in the gp1_gauntlet folder
    files = glob.glob(f"{dst_dir}/actor_trial_*.pth")

    # Extract the average monthly return value from the filenames and sort the files by their return values
    files_with_returns = [(f, float(re.findall(r"_return_([\d_]+)", f)[0].replace('_', '.'))) for f in files]
    files_with_returns.sort(key=lambda x: x[1], reverse=True)

    # Delete any files that are not in the top 10
    for i in range(10, len(files_with_returns)):
        os.remove(files_with_returns[i][0])
    
    return cumulative_monthly_loss, sharpe_ratio, cumulative_monthly_return

In [ ]:
study_name = "gp1_gauntlet"
directions = ["minimize", "maximize", "maximize"]

study = optuna.create_study(study_name=study_name, directions=directions)

# timeout=3600 is 1 hr, 86400 is 24hrs or n_trials=100
study.optimize(optuna_gauntlet_objective, timeout=(3600 * 24))

In [ ]:
optuna.visualization.plot_pareto_front(study, target_names=["cumulative_monthly_loss", "sharpe_ratio", "cumulative_monthly_return"])

In [ ]:
# Export Visualization
import plotly.io as pio


pareto_front = optuna.visualization.plot_pareto_front(study, target_names=["cumulative_monthly_loss", "sharpe_ratio", "cumulative_monthly_return"])

pio.write_html(pareto_front, file='pareto_front.html', auto_open=False)

In [ ]:
# trial_number = 79  # Choose a specific trial number.
# specific_trial = [trial for trial in study.trials if trial.number == trial_number][0]
# specific_hyperparameters = specific_trial.params
# print(f"Hyperparameters of trial number {trial_number}:", specific_hyperparameters)

# Part 3: Backtest

In [ ]:
ERL_PARAMS = {
    "learning_rate": 2e-5,
    "batch_size": 2048,
    "gamma":  0.985,
    "seed":42,
    "net_dimension":[128, 256], 
    "target_step":5000, 
    "eval_gap":50,
    "eval_times":1
}
env = StockTradingEnv

TEST_START_DATE = "2023-01-02"
TEST_END_DATE = "2023-03-31"

gp1_account_value_df=test(start_date = TEST_START_DATE, 
                    end_date = TEST_END_DATE,
                    ticker_list = ticker_list, 
                    data_source = 'alpaca',
                    time_interval= '1Min', 
                    technical_indicator_list= INDICATORS,
                    drl_lib='elegantrl', 
                    env=env, 
                    model_name='ppo',
                    if_vix=True,
                    if_cdl=True,
                    API_KEY = ALPACA_API_KEY, 
                    API_SECRET = ALPACA_API_SECRET, 
                    API_BASE_URL = API_BASE_URL,
                    cwd='./gp1_trained',
                    net_dimension = ERL_PARAMS['net_dimension'])

In [ ]:
print("==============Get Baseline Stats===========")

# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX

baseline_df = get_baseline(
        ticker="^GSPC", 
        start = TEST_START_DATE,
        end = TEST_END_DATE)

stats = backtest_stats(baseline_df, value_col_name = 'close')

In [ ]:
print("==============GP1 Stats===========")

stats = backtest_stats(gp1_account_value_df)

## Charts

In [ ]:
%matplotlib inline

gp1_account_value_df = drop_dup_dates(gp1_account_value_df)

# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX

backtest_plot(gp1_account_value_df, 
             baseline_ticker = '^GSPC', 
             baseline_start = TEST_START_DATE,
             baseline_end = TEST_END_DATE,)

# Part 4: Deploy the Agent

## Setup Alpaca Trading Environment

In [ ]:
import datetime
import threading
from processor_alpaca import AlpacaProcessor
import alpaca_trade_api as tradeapi
import time
import pandas as pd
import numpy as np
import torch
import gymnasium as gym

class AlpacaPaperTrading():

    def __init__(self,ticker_list, time_interval, drl_lib, agent, cwd, net_dim, 
                 state_dim, action_dim, API_KEY, API_SECRET, 
                 API_BASE_URL, tech_indicator_list, turbulence_thresh=99, 
                 max_stock=1e2, latency = None):
        #load agent
        self.drl_lib = drl_lib
        if agent =='ppo':
            if drl_lib == 'elegantrl':              
                agent_class = AgentPPO
                agent = agent_class(net_dim, state_dim, action_dim)
                actor = agent.act
                # load agent
                try:  
                    cwd = cwd + '/actor.pth'
                    print(f"| load actor from: {cwd}")
                    actor.load_state_dict(torch.load(cwd, map_location=lambda storage, loc: storage))
                    self.act = actor
                    self.device = agent.device
                except BaseException:
                    raise ValueError("Fail to load agent!")
                    
            else:
                raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
               
        else:
            raise ValueError('Agent input is NOT supported yet.')
            
            
            
        #connect to Alpaca trading API
        try:
            self.alpaca = tradeapi.REST(API_KEY,API_SECRET,API_BASE_URL, 'v2')
        except:
            raise ValueError('Fail to connect Alpaca. Please check account info and internet connection.')
        
        #read trading time interval
        if time_interval == '1s':
            self.time_interval = 1
        elif time_interval == '5s':
            self.time_interval = 5
        elif time_interval == '1Min':
            self.time_interval = 60
        elif time_interval == '5Min':
            self.time_interval = 60 * 5
        elif time_interval == '15Min':
            self.time_interval = 60 * 15
        else:
            raise ValueError('Time interval input is NOT supported yet.')
        
        #read trading settings
        self.tech_indicator_list = tech_indicator_list
        self.turbulence_thresh = turbulence_thresh
        self.max_stock = max_stock 
        
        #initialize account
        self.stocks = np.asarray([0] * len(ticker_list)) #stocks holding
        self.stocks_cd = np.zeros_like(self.stocks) 
        self.cash = None #cash record 
        self.stocks_df = pd.DataFrame(self.stocks, columns=['stocks'], index = ticker_list)
        self.assets_list = []
        self.price = np.asarray([0] * len(ticker_list))
        self.stockUniverse = ticker_list
        self.turbulence_bool = 0
        self.equities = []
        
    def test_latency(self, test_times = 10): 
        total_time = 0
        for i in range(0, test_times):
            time0 = time.time()
            self.get_state()
            time1 = time.time()
            temp_time = time1 - time0
            total_time += temp_time
        latency = total_time/test_times
        print('latency for data processing: ', latency)
        return latency
        
    def run(self):
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
          self.alpaca.cancel_order(order.id)
    
        # Wait for market to open.
        print("Waiting for market to open...")
        self.awaitMarketOpen()
        print("Market opened.")

        while True:

          # Figure out when the market will close so we can prepare to sell beforehand.
          clock = self.alpaca.get_clock()
          closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          self.timeToClose = closingTime - currTime
    
          if(self.timeToClose < (60)):
            # Close all positions when 1 minutes til market close.
            print("Market closing soon. Stop trading.")
            break
            
            '''# Close all positions when 1 minutes til market close.
            print("Market closing soon.  Closing positions.")

            threads = []
            positions = self.alpaca.list_positions()
            for position in positions:
              if(position.side == 'long'):
                orderSide = 'sell'
              else:
                orderSide = 'buy'
              qty = abs(int(float(position.qty)))
              respSO = []
              tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
              tSubmitOrder.start()
              threads.append(tSubmitOrder)    # record thread for joining later

            for x in threads:   #  wait for all threads to complete
                x.join()     
            # Run script again after market close for next trading day.
            print("Sleeping until market close (15 minutes).")
            time.sleep(60 * 15)'''
            
          else:
            self.trade()
            last_equity = float(self.alpaca.get_account().last_equity)
            cur_time = time.time()
            self.equities.append([cur_time,last_equity])
            time.sleep(self.time_interval)
            
    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open
        while(not isOpen):
          clock = self.alpaca.get_clock()
          openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          timeToOpen = int((openingTime - currTime) / 60)
          print(str(timeToOpen) + " minutes til market open.")
          time.sleep(60)
          isOpen = self.alpaca.get_clock().is_open
    
    def trade(self):
        state = self.get_state()
        
        if self.drl_lib == 'elegantrl':
            with torch.no_grad():
                s_tensor = torch.as_tensor((state,), device=self.device)
                a_tensor = self.act(s_tensor)  
                action = a_tensor.detach().cpu().numpy()[0]  
            action = (action * self.max_stock).astype(int)
            
        elif self.drl_lib == 'rllib':
            action = self.agent.compute_single_action(state)
        
        elif self.drl_lib == 'stable_baselines3':
            action = self.model.predict(state)[0]
            
        else:
            raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
        
        self.stocks_cd += 1
        if self.turbulence_bool == 0:
            min_action = 10  # stock_cd
            threads = []
            for index in np.where(action < -min_action)[0]:  # sell_index:
                sell_num_shares = min(self.stocks[index], -action[index])
                qty =  abs(int(sell_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'sell', respSO))
                tSubmitOrder.start()
                threads.append(tSubmitOrder)    # record thread for joining later
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0
            
            for x in threads:   #  wait for all threads to complete
                x.join()     

            threads = []
            for index in np.where(action > min_action)[0]:  # buy_index:
                if self.cash < 0:
                    tmp_cash = 0
                else:
                    tmp_cash = self.cash
                buy_num_shares = min(tmp_cash // self.price[index], abs(int(action[index])))
                if (buy_num_shares != buy_num_shares): # if buy_num_change = nan
                    qty = 0 # set to 0 quantity
                else:
                    qty = abs(int(buy_num_shares))
                qty = abs(int(buy_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'buy', respSO))
                tSubmitOrder.start()
                threads.append(tSubmitOrder)    # record thread for joining later
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0

            for x in threads:   #  wait for all threads to complete
                x.join()     
                
        else:  # sell all when turbulence
            threads = []
            positions = self.alpaca.list_positions()
            for position in positions:
                if(position.side == 'long'):
                    orderSide = 'sell'
                else:
                    orderSide = 'buy'
                qty = abs(int(float(position.qty)))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
                tSubmitOrder.start()
                threads.append(tSubmitOrder)    # record thread for joining later

            for x in threads:   #  wait for all threads to complete
                x.join()     
            
            self.stocks_cd[:] = 0
            
    
    def get_state(self):
        alpaca = AlpacaProcessor(api=self.alpaca)
        price, tech, turbulence = alpaca.fetch_latest_data(ticker_list = self.stockUniverse, time_interval='1Min',
                                                     tech_indicator_list=self.tech_indicator_list)
        turbulence_bool = 1 if turbulence >= self.turbulence_thresh else 0
        
        turbulence = (self.sigmoid_sign(turbulence, self.turbulence_thresh) * 2 ** -5).astype(np.float32)
        
        tech = tech * 2 ** -7
        positions = self.alpaca.list_positions()
        stocks = [0] * len(self.stockUniverse)
        for position in positions:
            ind = self.stockUniverse.index(position.symbol)
            stocks[ind] = ( abs(int(float(position.qty))))
        
        stocks = np.asarray(stocks, dtype = float)
        cash = float(self.alpaca.get_account().cash)
        self.cash = cash
        self.stocks = stocks
        self.turbulence_bool = turbulence_bool 
        self.price = price
        
        
        
        amount = np.array(self.cash * (2 ** -12), dtype=np.float32)
        scale = np.array(2 ** -6, dtype=np.float32)
        state = np.hstack((amount,
                    turbulence,
                    self.turbulence_bool,
                    price * scale,
                    self.stocks * scale,
                    self.stocks_cd,
                    tech,
                    )).astype(np.float32)
        state[np.isnan(state)] = 0.0
        state[np.isinf(state)] = 0.0
        print(len(self.stockUniverse))
        return state
        
    def submitOrder(self, qty, stock, side, resp):
        if(qty > 0):
          try:
            self.alpaca.submit_order(stock, qty, side, "market", "day")
            print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
            resp.append(True)
          except:
            print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
            resp.append(False)
        else:
          print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
          resp.append(True)

    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh

## Run Trading

In [ ]:
print(GP1_TICKER)

In [ ]:
state_dim

In [ ]:
action_dim

In [ ]:
ERL_PARAMS = {
    "learning_rate": 2e-5,
    "batch_size": 2048,
    "gamma":  0.985,
    "seed":42,
    "net_dimension":[128, 256], 
    "target_step":5000, 
    "eval_gap":50,
    "eval_times":1
}

In [ ]:
trading_erl = AlpacaPaperTrading(ticker_list = GP1_TICKER, 
                                       time_interval = '1Min', 
                                       drl_lib = 'elegantrl', 
                                       agent = 'ppo', 
                                       cwd = './gp1_trained', 
                                       net_dim = ERL_PARAMS['net_dimension'], 
                                       state_dim = state_dim, 
                                       action_dim= action_dim, 
                                       API_KEY = ALPACA_API_KEY, 
                                       API_SECRET = ALPACA_API_SECRET, 
                                       API_BASE_URL = API_BASE_URL, 
                                       tech_indicator_list = INDICATORS,
                                       max_stock=1e2, turbulence_thresh=99)
trading_erl.run()

# Part 5: Check Portfolio Performance

In [ ]:
import alpaca_trade_api as tradeapi
import exchange_calendars as tc
import numpy as np
import pandas as pd
import pytz
import yfinance as yf
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from datetime import datetime as dt
from plot import backtest_stats
import matplotlib.pyplot as plt

In [ ]:
def get_trading_days(start, end):
    nyse = tc.get_calendar('NYSE')
    df = nyse.sessions_in_range(pd.Timestamp(start,tz=pytz.UTC),
                                pd.Timestamp(end,tz=pytz.UTC))
    trading_days = []
    for day in df:
        trading_days.append(str(day)[:10])

    return trading_days

def alpaca_history(key, secret, url, start, end):
    api = tradeapi.REST(key, secret, url, 'v2')
    trading_days = get_trading_days(start, end)
    df = pd.DataFrame()
    for day in trading_days:
        df = df.append(api.get_portfolio_history(date_start = day,timeframe='5Min').df.iloc[:78])
    equities = df.equity.values
    cumu_returns = equities/equities[0]
    cumu_returns = cumu_returns[~np.isnan(cumu_returns)]
    
    return df, cumu_returns

def SP500_history(start):
    data_df = yf.download(['^GSPC'],start=start, interval="5m")
    data_df = data_df.iloc[:]
    baseline_returns = data_df['Adj Close'].values/data_df['Adj Close'].values[0]
    return data_df, baseline_returns

## Get cumulative return

In [ ]:
df_erl, cumu_erl = alpaca_history(key=ALPACA_API_KEY, 
                                  secret=ALPACA_API_SECRET, 
                                  url=API_BASE_URL, 
                                  start='2023-03-24', #must be within 1 month
                                  end='2023-03-24') #change the date if error occurs

In [ ]:
df_sp500, cumu_sp500 = SP500_history(start='2023-01-01')

In [ ]:
df_erl.tail()

In [ ]:
returns_erl = cumu_erl -1 
returns_sp500 = cumu_sp500 - 1
returns_sp500 = returns_sp500[:returns_erl.shape[0]]
print('len of erl return: ', returns_erl.shape[0])
print('len of sp500 return: ', returns_sp500.shape[0])

In [ ]:
returns_erl

## plot and save

In [ ]:
import matplotlib.pyplot as plt
plt.figure(dpi=1000)
plt.grid()
plt.grid(which='minor', axis='y')
plt.title('Stock Trading (Paper trading)', fontsize=20)
plt.plot(returns_erl, label = 'ElegantRL Agent', color = 'red')
#plt.plot(returns_sb3, label = 'Stable-Baselines3 Agent', color = 'blue' )
#plt.plot(returns_rllib, label = 'RLlib Agent', color = 'green')
plt.plot(returns_sp500, label = 'S&P 500', color = 'grey')
plt.ylabel('Return', fontsize=16)
plt.xlabel('Year 2023', fontsize=16)
plt.xticks(size = 14)
plt.yticks(size = 14)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(78))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.005))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=2))
ax.xaxis.set_major_formatter(ticker.FixedFormatter(['','10-19','','10-20',
                                                    '','10-21','','10-22']))
plt.legend(fontsize=10.5)
plt.savefig('papertrading_stock.png')